# Build Un-admissibile areas and boxes for coverage computations
- 1. Un-admissibile areas
- 2. Boxex


In [4]:
from shapely.geometry import Point, Polygon,box
import yaml
import geopandas as gpd
import cartopy.geodesic as GD
import geopandas as gpd
import pyproj
from functools import partial
import shapely
import numpy as np


with open('../conf.yaml') as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)
    
#used to parse the conf scales
def convert(s):
    try:
        return float(s)
    except ValueError:
        num, denom = s.split('/')
        return float(num) / float(denom)

pathBox = conf["pathBox"]
unadmissible_values = conf["unadmissible_values"]
unadmissible_values = (list(map(convert, unadmissible_values)))
unadmissible_circle_radius = conf["unadmissible_circle_radius"]
unadmissible_circle_area = unadmissible_circle_radius*unadmissible_circle_radius*np.pi
pathGeneral = conf["pathGeneral"]

print(unadmissible_values)
print(unadmissible_circle_radius)
print(unadmissible_circle_area)




[0.1, 0.3, 0.6]
1000
3141592.653589793


## Static un-admissibile areas
- version 1: write wide polygons for low, normal and high boxes
- version 2: write small polygons high box. The covered areas depends on the total area of the high box (e.g. 5%)

In [1]:
#draw un-admissibile areas version 1

#un-admissibile areas in boxLow
circle_points = GD.Geodesic().circle( lon= 116.074, lat=40.368, radius = 500)
circle_poly1 = Polygon(circle_points)
box1 = box(116.03,40.32,116.04,40.33)
poly1 = Polygon([(116.01,40.35),(116.01,40.39),(116.035,40.37)])

#un-admissibile areas in boxNormal
box2 = box(116.6,40.27,116.7,40.275)
poly2 = Polygon([(116.63,40.22),(116.642,40.27),(116.64,40.3)])
poly3 = Polygon([(116.6,40.28),(116.6,40.34),(116.67,40.34)])
circle_points = GD.Geodesic().circle( lon= 116.615, lat=40.25, radius = 750)
circle_poly2 = Polygon(circle_points)


#un-admissibile areas in boxHigh
box3 = box(116.39,39.95,116.4,39.97)
circle_points = GD.Geodesic().circle( lon= 116.35, lat=39.93, radius = 1000)
circle_poly3 = Polygon(circle_points)
box4 = box(116.38,39.9,116.40,39.92)
circle_points = GD.Geodesic().circle( lon= 116.31, lat=39.98, radius = 300)
circle_poly4 = Polygon(circle_points)
circle_points = GD.Geodesic().circle( lon= 116.37, lat=39.945, radius = 415)
circle_poly5 = Polygon(circle_points)

#scrittura su file
areeProibite = gpd.GeoSeries([circle_poly1,box1,poly1,box2,poly2,poly3,circle_poly2,box3,box4,circle_poly3,circle_poly4,circle_poly5],crs="EPSG:4326")
areeProibite.to_file(pathAreeProibite)


NameError: name 'GD' is not defined

## Dynamic un-admissible areas

In [5]:
#draw un-admissibile areas version 2
# percentage of the total area of the box (ex. 5% 10% 15%)
import warnings
warnings.filterwarnings("ignore")

def get_area(poly):
    geom_aea = shapely.ops.transform(partial(pyproj.transform,pyproj.Proj(init='EPSG:4326'),pyproj.Proj(proj='aea',lat_1=poly.bounds[1],lat_2=poly.bounds[3])),poly)
    return geom_aea.area

def get_valid_circle(circles, box, no_overlap = False):
    found = False
    if(no_overlap == False):
        lon = np.random.uniform(low=box.bounds[0],high= box.bounds[2])
        lat = np.random.uniform(low=box.bounds[1],high= box.bounds[3])
        circle = GD.Geodesic().circle( lon= lon, lat=lat, radius = unadmissible_circle_radius)
        circle = Polygon(circle)
        return circle
   # print("Without overalapping areas")
    while found==False:
        
        lon = np.random.uniform(low=box.bounds[0],high= box.bounds[2])
        lat = np.random.uniform(low=box.bounds[1],high= box.bounds[3])
        circle = GD.Geodesic().circle( lon= lon, lat=lat, radius = unadmissible_circle_radius)
        circle = Polygon(circle)
        if len(circles) == 0:
            return circle
        
        for c in circles:
            if c.overlaps(circle):
                found = False
                break
            else:
                found = True
    return circle

    
# Read the box high
boxes = gpd.GeoDataFrame(gpd.read_file(pathBox),crs="EPSG:4326")
boxes=boxes.set_index('index')

boxHigh=boxes.at["boxHigh",'geometry']
# compute the area as square meters
print("Area of boxHigh is: ",get_area(boxHigh))
print("Area of the circles: ",unadmissible_circle_area)


# x = lon, y = lat        
area = get_area(boxHigh)
for uvalue in unadmissible_values:
    print("processing ",uvalue)
    target_value = area*uvalue
    print("target area is: ",target_value)
    n_circles = round(target_value/unadmissible_circle_area)
    print("number of circles",n_circles)
    # build the first set of un-admissble circles
    circles = []
    for i in range(n_circles):
        #lon = np.random.uniform(low=boxHigh.bounds[0],high= boxHigh.bounds[2])
        #lat = np.random.uniform(low=boxHigh.bounds[1],high= boxHigh.bounds[3])
        #circle = GD.Geodesic().circle( lon= lon, lat=lat, radius = 100)
        #circle_poly = Polygon(circle)
        circle_poly = get_valid_circle(circles, boxHigh, True)
        circles.append(circle_poly)
    gpd.GeoSeries(circles,crs="EPSG:4326").to_file(pathGeneral+"unadmissible_area_"+str(uvalue)+".shp")
    a = 0
    for c in circles:
        a += c.area
    print("Area for ",uvalue,a)


Area of boxHigh is:  94885022.2577306
Area of the circles:  3141592.653589793
processing  0.1
target area is:  9488502.22577306
number of circles 3
Area for  0.1 0.0009932738123249296
processing  0.3
target area is:  28465506.67731918
number of circles 9
Area for  0.3 0.002979652502144316
processing  0.6
target area is:  56931013.35463836
number of circles 18
Area for  0.6 0.005958643689795052


## 2. Box generation for different scenario
- Low, High, Border, Full


In [ ]:
#scrittura box per test
boxLow = box(116,40.3,116.1,40.4)
boxHigh = box(116.3,39.9,116.4,40)
boxNormal = box(116.6,40.25,116.7,40.35)
boxBorder = box(116.2,39.95,116.3,40.05)
boxFull = box (116,40,116.1,40.1)

Box = gpd.GeoSeries(data=[boxLow,boxHigh,boxNormal,boxBorder,boxFull],index=["boxLow","boxHigh","boxNormal","boxBorder","boxFull"],crs="EPSG:4326")
Box.to_file(pathBox)
print(Box)